### Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Install and Import Libraries

In [ ]:
! pip install --quiet openpyxl

In [ ]:
import os
import json
import pandas as pd

### Initialize Test Config

In [ ]:
output_dir = "/content/drive/Shareddrives/CS198-Drones/[v5] Results"

evaluation_dir = "/content/drive/Shareddrives/CS198-Drones/[v5] Model Evaluation/"
results_output_file = f"{output_dir}/model_evaluation.xlsx"

training_dir = "/content/drive/Shareddrives/CS198-Drones/[v5] Training Output/"
training_output_file = f"{output_dir}/training_metrics.xlsx"

### Compile Evaluation Metrics

In [ ]:
# List of models in desired order
model_order = [
    "convnextv2-base-1k-224_rice-leaf-disease-augmented-v4_v5_fft",
    "vit-hybrid-base-bit-384_rice-leaf-disease-augmented-v4_v5_fft",
    "vit-base-patch16-224_rice-leaf-disease-augmented-v4_v5_fft",
    "swin-base-patch4-window7-224_rice-leaf-disease-augmented-v4_v5_fft",
    "deit-base-patch16-224_rice-leaf-disease-augmented-v4_v5_fft",
    "dinov2-base_rice-leaf-disease-augmented-v4_v5_fft",

    "vit_small_patch16_224.augreg_in21k_rice-leaf-disease-augmented-v4_v5_fft",
    "swin-tiny-patch4-window7-224_rice-leaf-disease-augmented-v4_v5_fft",
    "deit-small-patch16-224_rice-leaf-disease-augmented-v4_v5_fft",
    "convnextv2-tiny-1k-224_rice-leaf-disease-augmented-v4_v5_fft",

    "mobilevit-small_rice-leaf-disease-augmented-v4_v5_fft",
    "mobilevitv2_150.cvnets_in22k_ft_in1k_rice-leaf-disease-augmented-v4_v5_fft",
    "efficientnet-b2_rice-leaf-disease-augmented-v4_v5_fft",
    "efficientvit_b1.r224_in1k_rice-leaf-disease-augmented-v4_v5_fft",
    "efficientvit_m4.r224_in1k_rice-leaf-disease-augmented-v4_v5_fft",
    "efficientformer_l1.snap_dist_in1k_rice-leaf-disease-augmented-v4_v5_fft",
    "efficientformerv2_s2.snap_dist_in1k_rice-leaf-disease-augmented-v4_v5_fft",
]

model_name = [
    "ConvNeXtV2 Base",
    "ViT Hybrid Base",
    "ViT Base",
    "Swin Base",
    "DeiT Base",
    "DINOv2 Base",

    "ViT Small",
    "Swin Small",
    "DeiT Small",
    "ConvNeXtV2 Small",

    "MobileViT Small",
    "MobileViTv2 150",
    "EfficientNet B2",
    "EfficientViT B1",
    "EfficientViT M4",
    "EfficientFormer L1",
    "EfficientFormerV2 S2",
]

rename_map = dict(zip(model_order, model_name))

In [ ]:
def compile_evaluation_metrics(json_folder, output_file):
    print(f"\n⌛️ Compiling evaluation metrics from {json_folder}...")
    precision_data = []
    recall_data = []
    f1_data = []
    overall_data = []

    for model_dir in os.listdir(json_folder):
        print(f"\n⌛️ Processing {model_dir}...")
        model_path = os.path.join(json_folder, model_dir)
        json_file_path = os.path.join(model_path, "report.json")
        if os.path.isdir(model_path) and os.path.isfile(json_file_path):
            with open(json_file_path, "r") as f:
                data = json.load(f)
                model_name = model_dir

                for class_name, metrics in data.items():
                    if isinstance(metrics, dict):
                        precision_data.append({
                            "Model": model_name,
                            "Class": class_name,
                            "Precision": metrics.get("precision", None),
                        })
                        recall_data.append({
                            "Model": model_name,
                            "Class": class_name,
                            "Recall": metrics.get("recall", None),
                        })
                        f1_data.append({
                            "Model": model_name,
                            "Class": class_name,
                            "F1-Score": metrics.get("f1-score", None),
                        })

                # Add overall data
                weighted_ave = data["weighted avg"]
                overall_data.append({
                    "Model": model_name,
                    "Accuracy": data.get("accuracy", None),
                    "Precision": weighted_ave.get("precision", None),
                    "Recall": weighted_ave.get("recall", None),
                    "F1-Score": weighted_ave.get("f1-score", None),
                    "Support": weighted_ave.get("support", None),
                    "Eval Time": data.get("evaluation_time_sec", None),
                })

    print("\n⌛️ Saving evaluation metrics to Excel file...")
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        p_df = pd.DataFrame(precision_data).pivot(
            index="Model", columns="Class", values="Precision"
        )

        p_df = p_df.reindex(model_order)
        p_df.to_excel(writer, sheet_name="Precision")

        r_df = pd.DataFrame(recall_data).pivot(
            index="Model", columns="Class", values="Recall"
        )
        r_df = r_df.reindex(model_order)
        r_df.to_excel(writer, sheet_name="Recall")

        f1_df = pd.DataFrame(f1_data).pivot(
            index="Model", columns="Class", values="F1-Score"
        )
        f1_df = f1_df.reindex(model_order)
        f1_df.to_excel(writer, sheet_name="F1-Score")

        pd.DataFrame(overall_data).to_excel(
            writer, sheet_name="Overall Accuracy", index=False
        )

    print(f"\n✅ Evaluation metrics compiled and saved to {output_file}")

### Compile Training Metrics

In [ ]:
import os
import pandas as pd

def compile_training_metrics(metrics_folder, output_file):
    print(f"\n⌛️ Compiling training metrics from {metrics_folder}...")

    # Define metric categories (these will be used as sheet names)
    metric_names = [
        "loss", "grad_norm", "learning_rate", "eval_loss", "eval_accuracy",
        "eval_runtime", "eval_samples_per_second", "eval_steps_per_second",
        "gpu_vram_allocated_mb", "gpu_vram_reserved_mb", "timestamp"
    ]

    # Initialize a dictionary to hold DataFrames for each metric
    metrics_dict = {metric: pd.DataFrame() for metric in metric_names}

    # Process only models in the defined list, in order
    for model_dir in model_order:
        print(f"\n⌛️ Processing {model_dir}...")
        model_path = os.path.join(metrics_folder, model_dir)
        metrics_file = os.path.join(model_path, "training_metrics.xlsx")

        if os.path.isdir(model_path) and os.path.isfile(metrics_file):
            df = pd.read_excel(metrics_file)

            if "epoch" not in df.columns:
                print(f"Skipping {model_dir} (No 'epoch' column found)")
                continue

            # Add each metric to its respective dictionary entry
            for metric in metric_names:
                if metric in df.columns:
                    if metrics_dict[metric].empty:
                        metrics_dict[metric] = df[["epoch", metric]].copy()
                        metrics_dict[metric].rename(columns={metric: model_dir}, inplace=True)
                    else:
                        metrics_dict[metric] = pd.merge(
                            metrics_dict[metric],
                            df[["epoch", metric]].rename(columns={metric: model_dir}),
                            on="epoch",
                            how="outer"
                        )

    # Save to Excel
    print(f"\n⌛️ Saving compiled metrics to {output_file}...")
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        for metric, df in metrics_dict.items():
            if not df.empty:
                # Reorder columns (after epoch) to match the models list
                cols = ['epoch'] + [m for m in model_order if m in df.columns]
                df = df[cols]
                df = df.rename(columns=rename_map)
                df.to_excel(writer, sheet_name=metric, index=False)

    print(f"\n✅ Training metrics compiled and saved to {output_file}")

### Runner Function

In [ ]:
compile_evaluation_metrics(evaluation_dir, results_output_file)
compile_training_metrics(training_dir, training_output_file)